In [1]:
import os
import pandas as pd 
import shutil
import configparser

In [2]:
def getBandsIndexes(cat,bands):
    return [list(cat.columns).index(c)+1 for c in bands]

In [45]:
def makeConfig(dirCat:str,fileNameCat:str,idcolumn:str,zspeccolumn:str,
               features:list,expNum:int,dirConfigs:str,configName:str,
               dirPath:str='Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\Vis_UV_IR'):
    
    cat=pd.read_csv(os.path.join(dirCat,fileNameCat))
    headers=list(cat)
    idcolumnidx=str(int(headers.index('Seq')+1))
    zspeccolumnidx=str(int(headers.index(zspeccolumn)+1))
    featuresIdx=getBandsIndexes(cat,features)
    featuresIdxStr=','.join(str(x) for x in featuresIdx)
    nNeuronsLayer1=str(int(2*len(featuresIdx)+1))
    nNeuronsLayer2=str(int(len(featuresIdx)-1))

    newIni='exp'+str(expNum).zfill(3)+'_'+configName+'.ini'
    shutil.copyfile(os.path.join(dirConfigs,fileConfigExample),os.path.join(dirConfigs,newIni))
    parser=configparser.RawConfigParser()
    parser.read(os.path.join(dirConfigs,newIni))
    parser.set('Train Input Info','photoColumnIdxList',featuresIdxStr)
    parser.set('Train Input Info','dirPath',dirCat)
    parser.set('Train Input Info','idcolumnidx',idcolumnidx)
    parser.set('Train Input Info','zspeccolumnidx',zspeccolumnidx)
    parser.set('Train Input Info','name',fileNameCat)
    parser.set('Test Input Info','photoColumnIdxList',featuresIdxStr)
    parser.set('Test Input Info','name',fileNameCat)
    parser.set('Test Input Info','dirPath',dirCat)
    parser.set('Test Input Info','idcolumnidx',idcolumnidx)
    parser.set('Test Input Info','zspeccolumnidx',zspeccolumnidx)
    parser.set('Output Info','dirpath',
               dirPath+'\exp'+str(expNum).zfill(3))
    parser.set('Output Info','modelSubDirName','ex'+str(expNum).zfill(3))
    parser.set('Learning Info','nHiddenNeurons',nNeuronsLayer1+','+nNeuronsLayer2)
    parser.set('Train Output Info','nameSuffix','_'+fileNameCat)
    parser.set('Test Output Info','nameSuffix','_'+fileNameCat)
    parser.set('Log Info','nameSuffix','_'+fileNameCat)
    parser.set('Restart Info','nameSuffix','_'+fileNameCat)

    with open(os.path.join(dirConfigs,newIni), 'w') as configfile:
        parser.write(configfile)
    return

In [29]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR'
fileConfigExample=r'exXX_phz_z_ConfigExample.ini'
expNum=1
fileNameCat=r'05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors.csv'
idcolumn='Seq'
zspeccolumn='z_spec'
cat=pd.read_csv(os.path.join(dirCat,fileNameCat))

In [5]:
bands={'Vis':['B','V','r','ip','zpp'],
      'UV':['u'],
      'IR':['Y','J','H','Ks']}

In [6]:
for key,val in bands.items():
    blist=[b+'magap3' for b in val]
    bands[key]=blist

In [24]:
for key,val in bands.items():
    fset=val
    fsetName=key
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 


fsetName='Vis_UV'
fset=bands['Vis']+bands['UV']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='Vis_IR'
fset=bands['Vis']+bands['IR']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='Vis_UV_IR'
fset=bands['Vis']+bands['UV']+bands['IR']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

In [28]:
os.chdir('D:\\')

# 19.01.2021. Vis-UV-IR experiments
configs=os.listdir(dirConfigs)
configs=[conf for conf in configs if conf.startswith('exp0')]
for conf in configs:
    conf=os.path.join(dirConfigs,conf)
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp001_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp002_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_UV.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp003_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_IR.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp004_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp005_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_IR.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp006_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_IR.ini
Ready


In [8]:
### Identifying most important IR band
expNum=7

In [9]:
for band in bands['IR']:
    fsetName='Vis_UV_'+band
    fset=bands['Vis']+bands['UV']+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [10]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp007','exp008','exp009','exp010']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp007_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Ymagap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp008_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Jmagap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp009_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Hmagap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp010_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_Ksmagap3.ini
Ready


In [11]:
### With narrow bands

In [19]:
narrows=['IB574ap3','IA527ap3','IA624ap3','NB816ap3']
bandBasic=bands['Vis']+bands['UV']+['Hmagap3']

In [13]:
expNum

11

In [14]:
for band in narrows:
    fsetName='Vis_UV_'+band
    fset=bands['Vis']+bands['UV']+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [15]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp011','exp012','exp013','exp014']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp011_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_IB574ap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp012_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_IA527ap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp013_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_IA624ap3.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp014_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_NB816ap3.ini
Ready


In [20]:
expNum

15

In [22]:
for band in narrows:
    fsetName='Vis_UV_H_'+band.replace('ap3','')
    fset=bandBasic+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [23]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp015','exp016','exp017','exp018']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp015_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_H_IB574.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp016_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_H_IA527.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp017_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_H_IA624.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp018_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_H_NB816.ini
Ready


In [24]:
### With J band

In [25]:
expNum

19

In [26]:
bandBasic=bands['Vis']+bands['UV']+['Jmagap3']
for band in narrows:
    fsetName='Vis_UV_J_'+band.replace('ap3','')
    fset=bandBasic+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [27]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp019','exp020','exp021','exp022']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp019_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IB574.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp020_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA527.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp021_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA624.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp022_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_NB816.ini
Ready


In [28]:
### With colours

In [36]:
colors={'UV':['u-B'],
        'Vis':['B-V', 'V-r', 'r-ip', 'ip-zpp', 'zpp-Y'], 
        'IR':['Y-J','J-H','H-Ks']}

In [32]:
expNum=23

In [39]:
fsetName='color_Vis'
fset=colors['Vis']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='color_Vis_UV'
fset=colors['Vis']+colors['UV']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='color_Vis_IR'
fset=colors['Vis']+colors['IR']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

fsetName='color_Vis_UV_IR'
fset=colors['Vis']+colors['UV']+colors['IR']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName)
expNum+=1 

In [40]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp023','exp024','exp025','exp026']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp023_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_color_Vis.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp024_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_color_Vis_UV.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp025_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_color_Vis_IR.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR\exp026_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_color_Vis_UV_IR.ini
Ready


In [41]:
### With 'dirty' dataset

In [42]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05\NoQfClean'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\dirtySample'
fileConfigExample=r'exXX_phz_z_ConfigExample.ini'
fileNameCatDirty=r'05_COSMOS_SpectrZ_Zlim_1p2_specZOutlCoeff_filter.csv'
fileNameCatRandom=r'05_COSMOS_SpectrZ_Zlim_1p2_randSample.csv'
dirPath='Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\DirtySample'
catDirty=pd.read_csv(os.path.join(dirCat,fileNameCatDirty))
catRandom=pd.read_csv(os.path.join(dirCat,fileNameCatRandom))

In [47]:
fsetName='Vis_UV_IR'
fset=bands['Vis']+bands['UV']+bands['IR']
makeConfig(dirCat,fileNameCatDirty,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCatDirty.replace('.csv','')+'_'+fsetName,dirPath=dirPath)
expNum+=1 

makeConfig(dirCat,fileNameCatRandom,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCatRandom.replace('.csv','')+'_'+fsetName,dirPath=dirPath)
expNum+=1 

In [48]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp027','exp028']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\dirtySample\exp027_05_COSMOS_SpectrZ_Zlim_1p2_specZOutlCoeff_filter_Vis_UV_IR.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\dirtySample\exp028_05_COSMOS_SpectrZ_Zlim_1p2_randSample_Vis_UV_IR.ini
Ready


In [49]:
### Feature selection based on correlation coefficient

In [52]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\FeatureSelection'
fileConfigExample=r'exXX_phz_z_ConfigExample.ini'
expNum=29
fileNameCat=r'05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors.csv'
idcolumn='Seq'
zspeccolumn='z_spec'
cat=pd.read_csv(os.path.join(dirCat,fileNameCat))
dirPath='Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\FeatureSelection'

In [63]:
fsetName='FS_Corr'
fset=['IB574ap3', 'Vmagap3', 'IA527ap3', 'IA624ap3', 'IB505ap3', 
      'rmagap3', 'IA484ap3', 'IA679ap3', 'r-ip', 'IB464ap3']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName,dirPath=dirPath)
expNum+=1 

In [64]:
os.chdir('D:\\')

configs=os.listdir(dirConfigs)
confIds=['exp029']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\FeatureSelection\exp029_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_FS_Corr.ini
Ready


In [66]:
### For the features chosen with RFE
fsetName='FS_RFE'
fset=['Vmagap3', 'IA527ap3', 'IA624ap3', 'IA738ap3', 'IB574ap3', 'u-B', 'B-V',
       'r-ip', 'ip-zpp', 'H-Ks']
makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
           features=fset,expNum=expNum,dirConfigs=dirConfigs,
               configName=fileNameCat.replace('.csv','')+'_'+fsetName,dirPath=dirPath)
expNum+=1 

In [67]:
configs=os.listdir(dirConfigs)
confIds=['exp030']
for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\FeatureSelection\exp030_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_FS_RFE.ini
Ready


In [74]:
### Systematic narrow bands with J band
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows'
fileConfigExample=r'exXX_phz_z_ConfigExample.ini'
fileNameCat=r'05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors.csv'
idcolumn='Seq'
zspeccolumn='z_spec'
cat=pd.read_csv(os.path.join(dirCat,fileNameCat))
dirPath=r'Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\Narrows'
expNum=31

In [75]:
bandBasic=bands['Vis']+bands['UV']+['Jmagap3']
narrows=[c for c in cat.columns if ('ap3' in c and 'mag' not in c)]
for band in narrows:
    fsetName='Vis_UV_J_'+band.replace('ap3','')
    fset=bandBasic+[band]
    makeConfig(dirCat,fileNameCat,idcolumn='Seq',zspeccolumn=zspeccolumn,
               features=fset,expNum=expNum,dirConfigs=dirConfigs,
                   configName=fileNameCat.replace('.csv','')+'_'+fsetName)
    expNum+=1 

In [77]:
confIds=['exp'+str(e).zfill(3) for e in range(31,expNum)]
print(confIds)

['exp031', 'exp032', 'exp033', 'exp034', 'exp035', 'exp036', 'exp037', 'exp038', 'exp039', 'exp040', 'exp041', 'exp042', 'exp043', 'exp044']


In [78]:
os.chdir('D:\\')

for confID in confIds:
    conf=os.path.join(dirConfigs,next(f for f in os.listdir(dirConfigs) if f.startswith(confID)))
    print(conf)
    os.system('metaphor_pipeline '+conf)
    print('Ready')

D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp031_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA484.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp032_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA527.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp033_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA624.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp034_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA679.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp035_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA738.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp036_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IA767.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Narrows\exp037_05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors_Vis_UV_J_IB427.ini
Ready
D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Nar

In [80]:
cat['yH']

KeyError: 'yH'